## the Consumer and Producer threads runs indefinitely while checking the status of the queue. 
## The Producer thread is responsible for putting items into the queue if it is not full 
## while the Consumer thread consumes items if there are any.

In [1]:
import threading
import time
import logging
import random
import Queue


logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)

BUF_SIZE = 10
q = Queue.Queue(BUF_SIZE)

class ProducerThread(threading.Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs=None, verbose=None):
       # super(ProducerThread,self).__init__()
        self.target = target
        self.name = name

    def run(self):
        while True:
            if not q.full():
                item = random.randint(1,10)
                q.put(item)
                logging.debug('Putting ' + str(item)  
                              + ' : ' + str(q.qsize()) + ' items in queue')
                time.sleep(random.random())
        return

class ConsumerThread(threading.Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs=None, verbose=None):
        #super(ConsumerThread,self).__init__()
        self.target = target
        self.name = name
        return

    def run(self):
        while True:
            if not q.empty():
                item = q.get()
                logging.debug('Getting ' + str(item) 
                              + ' : ' + str(q.qsize()) + ' items in queue')
                time.sleep(random.random())
        return

if __name__ == '__main__':
    
    p = ProducerThread(name='producer')
    c = ConsumerThread(name='consumer')

    p.start()
    time.sleep(2)
    c.start()
    time.sleep(2)

ModuleNotFoundError: No module named 'Queue'

In [2]:
pip install queue

ERROR: Could not find a version that satisfies the requirement queue (from versions: none)
ERROR: No matching distribution found for queue
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pip

     |████████████████████████████████| 1.5 MB 4.7 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.2
    Uninstalling pip-20.2.2:
      Successfully uninstalled pip-20.2.2
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install queue

ERROR: Could not find a version that satisfies the requirement queue
ERROR: No matching distribution found for queue
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install Queue

ERROR: Could not find a version that satisfies the requirement Queue
ERROR: No matching distribution found for Queue
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sys
import random
import time
from threading import *

class Producer(Thread):
	def __init__(self, items, can_produce, can_consume):
		Thread.__init__(self)
		self.items = items
		self.can_produce = can_produce
		self.can_consume = can_consume

	def produce_item(self):
		self.items.append(1)
		print "{}: i produced an item".format(self.name)

	def wait(self):
		time.sleep(random.uniform(0, 3))

	def run(self):
		while 1:
			self.wait()
			self.can_produce.acquire()
			self.produce_item()
			self.can_consume.release()	

class Consumer(Thread):
	def __init__(self, items, can_produce, can_consume):
		Thread.__init__(self)
		self.items = items
		self.can_produce = can_produce
		self.can_consume = can_consume

	def consume_item(self):
		item = self.items.pop()
		print "{}: i consumed an item".format(self.name)

	def wait(self):
		time.sleep(random.uniform(0, 3))

	def run(self):
		while 1:
			self.wait()
			self.can_consume.acquire()
			self.consume_item()
			self.can_produce.release()

def usage(script):
	print "Usage:\t%s count_producers count_consumers buffer_length" % script

if __name__ == "__main__":

	if len(sys.argv) != 4:
		usage(sys.argv[0])
		sys.exit(0)

	count_producers = int(sys.argv[1])
	count_consumers = int(sys.argv[2])
	buffer_length = int(sys.argv[3])

	items = []
	producers = []
	consumers = []

	#acquire while buffer is not full
	can_produce = Semaphore(buffer_length)

	#acquire while buffer is not empty
	can_consume = Semaphore(0)

	for i in range(count_producers):
		producers.append(Producer(items, can_produce, can_consume))
		producers[-1].start()

	for i in range(count_consumers):
		consumers.append(Consumer(items, can_produce, can_consume))
		consumers[-1].start()

	for p in producers:
		p.join()

	for c in consumers:
		c.join()